# Fragrance Segmentation – Unsupervised Clustering Analysis

_Course: Machine Learning_

- Name: Muhammad Umar Uz Zaman  
- Student ID: STU1197819  
- Goal: I explore a large fragrance dataset to understand:
  - How perfumes naturally group together based on their scent notes and key characteristics
  - Which fragrance segments tend to be more affordable versus more premium or luxury
  - How these segments can support targeted marketing, product positioning, and personalised recommendations
- Dataset: `Fragrance Dataset 4037.csv` (records of perfumes with information on notes and other descriptive attributes)

**Phases of this notebook**

Intro & setup  
Phase 1 – Data loading & structure check  
Phase 2 – Data cleaning, preparation & feature selection  
Phase 3 – Core EDA (distributions, relationships & feature importance)  
Phase 4 – Clustering models (K-Means, Hierarchical, DBSCAN) & evaluation  
Phase 5 – Cluster profiling & business insights  
Phase 6 – Wrap-up, limitations & export for management report


In [1]:
# Core libraries
import numpy as np
import pandas as pd

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Clustering and preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Hierarchical clustering tools for dendrograms
from scipy.cluster.hierarchy import dendrogram, linkage

# Plotting style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.titlesize"] = 12
plt.rcParams["axes.labelsize"] = 11
plt.rcParams["figure.dpi"] = 100


ModuleNotFoundError: No module named 'sklearn'